## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-07-16-51-14 +0000,nyt,How Families Are Coping with SNAP Cuts During ...,https://www.nytimes.com/2025/11/07/us/politics...
1,2025-11-07-16-50-12 +0000,nyt,New York Prepares for a Potential Trump Immigr...,https://www.nytimes.com/2025/11/07/nyregion/ny...
2,2025-11-07-16-49-00 +0000,wsj,Gold Prices Pull Back Slightly But Softer Doll...,https://www.wsj.com/finance/commodities-future...
3,2025-11-07-16-48-46 +0000,nyt,U.S. Wants Security Council to Adopt Trump Pla...,https://www.nytimes.com/2025/11/07/world/middl...
4,2025-11-07-16-46-12 +0000,nyt,When a Vietnam Protest on Campus Turned Deadly,https://www.nytimes.com/2025/11/07/us/kent-sta...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2323/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
10,trump,49
5,shutdown,17
6,new,16
127,year,13
92,flight,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
84,2025-11-07-13-30-00 +0000,wsj,"Republican Rep. Elise Stefanik, an ally of Pre...",https://www.wsj.com/politics/elections/elise-s...,138
97,2025-11-07-12-40-35 +0000,cbc,Trump administration fights court ruling to pa...,https://www.cbc.ca/news/world/us-ri-court-ruli...,117
394,2025-11-06-17-38-00 +0000,wsj,Many GOP lawmakers are saying no—for now—to Pr...,https://www.wsj.com/politics/policy/senate-rep...,112
224,2025-11-07-02-00-00 +0000,wsj,Allies of President Trump and Health Secretary...,https://www.wsj.com/politics/elections/trump-r...,107
240,2025-11-07-01-28-00 +0000,wsj,Trump Administration Must Fund Full SNAP Benef...,https://www.wsj.com/politics/policy/trump-admi...,104


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
84,138,2025-11-07-13-30-00 +0000,wsj,"Republican Rep. Elise Stefanik, an ally of Pre...",https://www.wsj.com/politics/elections/elise-s...
308,86,2025-11-06-22-09-16 +0000,nypost,Tesla shareholders approve Elon Musk’s histori...,https://nypost.com/2025/11/06/business/tesla-s...
39,76,2025-11-07-15-21-00 +0000,wsj,Former House Speaker Nancy Pelosi says she won...,https://www.wsj.com/politics/nancy-pelosi-reti...
162,72,2025-11-07-10-00-00 +0000,wsj,Democrats sought information about Social Secu...,https://www.wsj.com/politics/policy/senate-dem...
69,71,2025-11-07-14-11-00 +0000,wsj,Senate Majority Leader John Thune expects Sena...,https://www.wsj.com/politics/policy/government...
133,67,2025-11-07-11-00-00 +0000,latimes,Your flight was canceled amid FAA's cuts to ai...,https://www.latimes.com/california/story/2025-...
314,49,2025-11-06-21-54-53 +0000,nypost,Fed judge rules Trump admin must fund SNAP ben...,https://nypost.com/2025/11/06/us-news/fed-judg...
338,48,2025-11-06-20-54-00 +0000,wsj,Employers said they cut more than a million jo...,https://www.wsj.com/economy/jobs/planned-u-s-j...
211,45,2025-11-07-02-31-35 +0000,nypost,James Carville says he’d bet money Dems will w...,https://nypost.com/2025/11/06/us-news/james-ca...
103,43,2025-11-07-12-04-25 +0000,nypost,Disgraced ex-Rep. George Santos claims he’s fl...,https://nypost.com/2025/11/07/us-news/george-s...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
